[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rycroft-group/math714/blob/main/i_fvolume/fvolume.ipynb)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from math import exp, pi, sin, cos
import sys

# Optional: a library for plotting with LaTeX-like 
# styles nicer formatted figures
# Warning: need to have LaTeX installed
import scienceplots
plt.style.use(['science'])

# Finite volume method

## Comparison of numerical flux conservation

### Setup

In [ ]:
# Grid size
m = 256
u = np.empty((m))
v = np.empty((m))
r = np.empty((m))
s = np.empty((m))
snaps = 100
iters = 1000
z = np.empty((m, snaps+1))

# Spatially varying diffusivity
def beta(x):
    return 0.12+0.08*sin(2*pi*x)

# Initial condition
def f(x):
    if x < 0.25 or x > 0.75:
        return 0
    else:
        return 1


# PDE-related constants
dx = 1.0/m
dt = 0.3*dx*dx/(0.2*2)
mu = dt/(dx*dx)

# Set up initial condition
for i in range(m):
    u[i] = f(dx*i)
z[:, 0] = u

### Finite difference

In [ ]:
def finite_difference(m, snaps, iters, u, v, r, s, z, dx, mu):

    # Initialize r_i and s_i terms in finite difference stencil
    for i in range(m):
        x = dx*i
        A = 0.25*(beta(x+dx)-beta(x-dx))
        B = beta(x)
        r[i] = mu*(-A+B)
        s[i] = mu*(A+B)

    # Integrate the PDE
    p=1-r-s
    for i in range(1,snaps+1):
        for k in range(iters):
            v[:]=p*u+r*np.roll(u,1)+s*np.roll(u,-1)

            # Cycle pointers to the three arrays
            w=u;u=v;v=w
        z[:,i]=u

    return z

### Finite volume

In [ ]:
def finite_volume(m, snaps, iters, u, v, r, s, z, dx, mu):

    # Initialize r_i and s_i terms in finite volume stencil
    for i in range(m):
        x = dx*i
        r[i] = mu*beta(x-0.5*dx)
        s[i] = mu*beta(x+0.5*dx)

    # Integrate the PDE
    p = 1-r-s
    for i in range(1, snaps+1):
        for k in range(iters):
            v[:] = p*u+r*np.roll(u, 1)+s*np.roll(u, -1)

            # Cycle pointers to the three arrays
            w = u
            u = v
            v = w
        z[:, i] = u

    return z

### Compute and output

In [ ]:
z_fd = finite_difference(m, snaps, iters, u.copy(), v.copy(), r.copy(), s.copy(), z.copy(), dx, mu)
z_fv = finite_volume(m, snaps, iters, u.copy(), v.copy(), r.copy(), s.copy(), z.copy(), dx, mu)

In [ ]:
fd_sol = []
for j in range(m):
    fd_sol = [str(j*dx)]
    for i in range(snaps+1):
        fd_sol.append(str(z_fd[j, i]))
    print(" ".join(fd_sol))

fd_int = np.sum(z_fd[:,0])
fd_int_changes = []
for j in range(snaps+1):
    print(j, j*dt*iters, dx*(np.sum(z_fd[:,j]) - fd_int))
    fd_int_changes.append((j, j*dt*iters, dx*(np.sum(z_fd[:,j]) - fd_int)))

fv_sol = []
for j in range(m):
    fv_sol = [str(j*dx)]
    for i in range(snaps+1):
        fv_sol.append(str(z_fv[j, i]))
    print(" ".join(fv_sol))

fv_int = np.sum(z_fv[:,0])
fv_int_changes = []
for j in range(snaps+1):
    print(j, j*dt*iters, dx*(np.sum(z_fv[:,j]) - fv_int))
    fv_int_changes.append((j, j*dt*iters, dx*(np.sum(z_fv[:,j]) - fv_int)))

### Visualize

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4), dpi=300)

# Panel 1: Solution
x = np.arange(m) * dx

for i in range(snaps + 1):
    axes[0].plot(x, z_fd[:, i], color='tab:blue', alpha=0.2, linewidth=0.8)
    axes[0].plot(x, z_fv[:, i], color='tab:orange', alpha=0.2, linewidth=0.8)

axes[0].plot(x, z_fd[:, -1], label='Finite difference', color='tab:blue')
axes[0].plot(x, z_fv[:, -1], label='Finite volume', color='tab:orange')
axes[0].set_xlabel('$x$')
axes[0].set_ylabel('$q(x)$')
axes[0].legend()

# Panel 2: Change in integral over time
t = np.array(fv_int_changes)[:,1]
abs_fd_dint = np.abs(np.array(fd_int_changes)[:,2].astype(float))
abs_fv_dint = np.abs(np.array(fv_int_changes)[:,2].astype(float))
axes[1].scatter(t, abs_fd_dint, s=2, marker='X', label='Finite difference', color='tab:blue')
axes[1].scatter(t, abs_fv_dint, s=2, marker='X', label='Finite volume', color='tab:orange')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Absolute change in $\int_0^1 q(x,t) \, dx$')
axes[1].legend()
axes[1].set_yscale('log')
axes[1].set_ylim([1e-18, 1e-4])

plt.tight_layout()
plt.show()

## Higher-order finite volume method

### Setup

In [ ]:
# Grid size
m = 256
u = np.empty((m))
v = np.empty((m))
snaps = 64
iters = 20
z = np.empty((m, snaps+1))

# Initial condition
def f(x):
    # return exp(-20*(x-0.5)**2)
    if x > 0.1 and x < 0.6:
        return 1
    else:
        return 0

# PDE-related constants
A = 1.0
dx = 1.0/m
dt = 0.2*dx

# Set up initial condition
for i in range(m):
    u[i] = f(dx*i)
z[:, 0] = u

### Finite volume

In [ ]:
def finite_volume(method, A, dt, dx, m, snaps, iters, u, v, z):

    f = A*dt/dx

    # First-order Godunov method
    if method == "0":
        for i in range(1, snaps+1):
            for k in range(iters):
                v[:] = u+f*(np.roll(u, 1)-u)
                u, v = v, u
            z[:, i] = u

    # Lax-Wendroff method
    elif method == "1":
        sl = 0.5*f*(1+f)
        sc = 1-f*f
        sr = 0.5*f*(-1+f)
        for i in range(1, snaps+1):
            for k in range(iters):
                v[:] = sl*np.roll(u, 1)+sc*u+sr*np.roll(u, -1)
                u, v = v, u
            z[:, i] = u

    # Beam-Warming method
    elif method == "2":
        sll = -0.5*f+0.5*f*f
        sl = f*(2-f)
        sc = 1-1.5*f+0.5*f*f
        for i in range(1, snaps+1):
            for k in range(iters):
                v[:] = sll*np.roll(u, 2)+sl*np.roll(u, 1)+sc*u
                u, v = v, u
            z[:, i] = u

    else:
        print("Integration type not known")
        sys.exit()

    # Output results
    results = []
    for j in range(m):
        e = [str(j*dx)]
        for i in range(snaps+1):
            e.append(str(z[j, i]))
        print(" ".join(e))
        results.append(e)

    return results

### Compute and output

In [ ]:
z_godunov = finite_volume("0", A, dt, dx, m, snaps, iters, u.copy(), v.copy(), z.copy())
z_lxw = finite_volume("1", A, dt, dx, m, snaps, iters, u.copy(), v.copy(), z.copy())
z_bw = finite_volume("2", A, dt, dx, m, snaps, iters, u.copy(), v.copy(), z.copy())

### Visualize

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), dpi=300)
snap_indices = np.arange(0, 20, 4)
colors = plt.cm.viridis(np.linspace(0, 1, len(snap_indices)))

# Godunov
x_godunov = np.array([float(row[0]) for row in z_godunov])
for idx, color in zip(snap_indices, colors):
    y_godunov = np.array([float(row[idx + 1]) for row in z_godunov])
    ax.plot(x_godunov, y_godunov, color=color, linestyle='-', alpha=0.7)

# Lax-Wendroff
# x_lxw = np.array([float(row[0]) for row in z_lxw])
# for idx, color in zip(snap_indices, colors):
#     y_lxw = np.array([float(row[idx + 1]) for row in z_lxw])
#     ax.plot(x_lxw, y_lxw, color=color, linestyle='--', alpha=0.7)

# Beam-Warming
# x_bw = np.array([float(row[0]) for row in z_bw])
# for idx, color in zip(snap_indices, colors):
#     y_bw = np.array([float(row[idx + 1]) for row in z_bw])
#     ax.plot(x_bw, y_bw, color=color, linestyle=':', alpha=0.7)

ax.set_xlabel('$x$')
ax.set_ylabel('$q(x)$')
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 4), dpi=300)
snap_indices = np.arange(0, 20, 4)
colors = plt.cm.viridis(np.linspace(0, 1, len(snap_indices)))

# Godunov
x_godunov = np.array([float(row[0]) for row in z_godunov])
for idx, color in zip(snap_indices, colors):
    y_godunov = np.array([float(row[idx + 1]) for row in z_godunov])
    axes[0].plot(x_godunov, y_godunov, color=color, alpha=0.7, label=f't={idx}')
axes[0].set_title('Godunov')
axes[0].set_xlabel('$x$')
axes[0].set_ylabel('$q(x)$')
axes[0].set_ylim([-0.3, 1.3])

# Lax-Wendroff
x_lxw = np.array([float(row[0]) for row in z_lxw])
for idx, color in zip(snap_indices, colors):
    y_lxw = np.array([float(row[idx + 1]) for row in z_lxw])
    axes[1].plot(x_lxw, y_lxw, color=color, alpha=0.7, label=f't={idx}')
axes[1].set_title('Lax-Wendroff')
axes[1].set_xlabel('$x$')
axes[1].set_ylabel('$q(x)$')
axes[1].set_ylim([-0.3, 1.3])

# Beam-Warming
x_bw = np.array([float(row[0]) for row in z_bw])
for idx, color in zip(snap_indices, colors):
    y_bw = np.array([float(row[idx + 1]) for row in z_bw])
    axes[2].plot(x_bw, y_bw, color=color, alpha=0.7, label=f't={idx}')
axes[2].set_title('Beam-Warming')
axes[2].set_xlabel('$x$')
axes[2].set_ylabel('$q(x)$')
axes[2].set_ylim([-0.3, 1.3])

plt.tight_layout()
plt.show()

## Limiters

### Setup

In [ ]:
# Grid size
m = 256
u = np.empty((m))
v = np.empty((m))
snaps = 64
iters = 20
z = np.empty((m, snaps+1))

# Initial condition
def f(x):
    # return exp(-100*(x-0.5)**2)
    if x > 0.1 and x < 0.6:
        return 1
    else:
        return 0

# PDE-related constants
A = 1.0
dx = 1.0/m
dt = 0.2*dx

# Set up initial condition
for i in range(m):
    u[i] = f(dx*i)
z[:, 0] = u

# Define minmod and maxmod functions, used for slope limiter calculations
def minmod(a, b):
    if a*b <= 0:
        return 0
    elif abs(a) < abs(b):
        return a
    else:
        return b

def maxmod(a, b):
    if a*b <= 0:
        return 0
    elif abs(a) < abs(b):
        return b
    else:
        return a

### Finite volume + limiters

In [ ]:
def finite_volume_limiter(limiter, A, dt, dx, m, snaps, iters, u, v, z):

    f = A*dt/dx

    for i in range(1, snaps+1):
        for k in range(iters):

            # Compute limited slopes
            sl = (u-np.roll(u, 1))
            sr = np.roll(sl, -1)

            # Minmod limiters
            if limiter == "0":
                for j in range(m):
                    sl[j] = minmod(sl[j], sr[j])

            # Superbee limiters
            elif limiter == "1":
                for j in range(m):
                    sl[j] = maxmod(minmod(2*sr[j], sl[j]),
                                   minmod(sr[j], 2*sl[j]))

            else:
                print("Unknown mode")
                sys.exit()

            # Compute update using the limited slopes
            v[:] = u+f*(np.roll(u, 1)-u)-0.5*f*(1-f)*(sl-np.roll(sl, 1))
            u, v = v, u
        z[:, i] = u

    # Output results
    results = []
    for j in range(m):
        e = [str(j*dx)]
        for i in range(snaps+1):
            e.append(str(z[j, i]))
        print(" ".join(e))
        results.append(e)

    return results

### Compute and output

In [ ]:
z_mm = finite_volume_limiter("0", A, dt, dx, m, snaps, iters, u.copy(), v.copy(), z.copy())
z_sb = finite_volume_limiter("1", A, dt, dx, m, snaps, iters, u.copy(), v.copy(), z.copy())

### Visualize

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4), dpi=300)
snap_indices = np.arange(0, 20, 4)
colors = plt.cm.viridis(np.linspace(0, 1, len(snap_indices)))

# Minmod
x_minmod = np.array([float(row[0]) for row in z_mm])
for idx, color in zip(snap_indices, colors):
    y_minmod = np.array([float(row[idx + 1]) for row in z_mm])
    axes[0].plot(x_minmod, y_minmod, color=color, alpha=0.7)
axes[0].set_title('Minmod')
axes[0].set_xlabel('$x$')
axes[0].set_ylabel('$q(x)$')
axes[0].set_ylim([-0.3, 1.3])

# Superbee
x_superbee = np.array([float(row[0]) for row in z_sb])
for idx, color in zip(snap_indices, colors):
    y_superbee = np.array([float(row[idx + 1]) for row in z_sb])
    axes[1].plot(x_superbee, y_superbee, color=color, alpha=0.7)
axes[1].set_title('Superbee')
axes[1].set_xlabel('$x$')
axes[1].set_ylabel('$q(x)$')
axes[1].set_ylim([-0.3, 1.3])

# Godunov
x_godunov = np.array([float(row[0]) for row in z_godunov])
for idx, color in zip(snap_indices, colors):
    y_godunov = np.array([float(row[idx + 1]) for row in z_godunov])
    axes[2].plot(x_godunov, y_godunov, color=color, alpha=0.7)
axes[2].set_title('Godunov')
axes[2].set_xlabel('$x$')
axes[2].set_ylabel('$q(x)$')
axes[2].set_ylim([-0.3, 1.3])

plt.tight_layout()
plt.show()

## ENO2

### Setup

In [ ]:
# Grid size
m = 256
u = np.empty((m))
v = np.empty((m))
snaps = 64
iters = 20
z = np.empty((m, snaps+1))

# Initial condition
def f(x):
    if x > 0.1 and x < 0.6:
        return 1
    else:
        return 0

# PDE-related constants
A = 1.0
dx = 1.0/m
dt = 0.2*dx

# Set up initial condition
for i in range(m):
    u[i] = f(dx*i)
z[:, 0] = u

# Define ENO2 operation
def eno2(a, b, c, d):
    if abs(a-2*b+c) < abs(b-2*c+d):
        return 3*c-4*b+a
    else:
        return d-b

# Integrate the equation
f = 0.5*A*dt/dx
for i in range(1, snaps+1):
    for k in range(iters):
        ua = np.roll(u, 2)
        ub = np.roll(u, 1)
        ud = np.roll(u, -1)
        for j in range(m):
            v[j] = u[j]-f*eno2(ua[j], ub[j], u[j], ud[j])
        u, v = v, u
    z[:, i] = u

# Output results
results = []
for j in range(m):
    e = [str(j*dx)]
    for i in range(snaps+1):
        e.append(str(z[j, i]))
    print(" ".join(e))
    results.append(e)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), dpi=300)
snap_indices = np.arange(0, 20, 4)
colors = plt.cm.viridis(np.linspace(0, 1, len(snap_indices)))

x_eno2 = np.arange(m) * dx
for idx, color in zip(snap_indices, colors):
    y_eno2 = z[:, idx]
    ax.plot(x_eno2, y_eno2, color=color, alpha=0.7, label=f't={idx}')

ax.set_title('ENO2')
ax.set_xlabel('$x$')
ax.set_ylabel('$q(x)$')
ax.set_ylim([-0.3, 1.3])
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4), dpi=300)
snap_indices = np.arange(0, 20, 4)
colors = plt.cm.viridis(np.linspace(0, 1, len(snap_indices)))

# Minmod
x_minmod = np.array([float(row[0]) for row in z_mm])
for idx, color in zip(snap_indices, colors):
    y_minmod = np.array([float(row[idx + 1]) for row in z_mm])
    axes[0].plot(x_minmod, y_minmod, color=color, alpha=0.7, label=f't={idx}')
axes[0].set_title('Minmod')
axes[0].set_xlabel('$x$')
axes[0].set_ylabel('$q(x)$')
axes[0].set_ylim([-0.3, 1.3])

# Superbee
x_superbee = np.array([float(row[0]) for row in z_sb])
for idx, color in zip(snap_indices, colors):
    y_superbee = np.array([float(row[idx + 1]) for row in z_sb])
    axes[1].plot(x_superbee, y_superbee, color=color, alpha=0.7, label=f't={idx}')
axes[1].set_title('Superbee')
axes[1].set_xlabel('$x$')
axes[1].set_ylabel('$q(x)$')
axes[1].set_ylim([-0.3, 1.3])

# ENO2
x_eno2 = np.arange(m) * dx
for idx, color in zip(snap_indices, colors):
    y_eno2 = z[:, idx]
    axes[2].plot(x_eno2, y_eno2, color=color, alpha=0.7, label=f't={idx}')
axes[2].set_title('ENO2')
axes[2].set_xlabel('$x$')
axes[2].set_ylabel('$q(x)$')
axes[2].set_ylim([-0.3, 1.3])

plt.tight_layout()
plt.show()

## Vanishing viscosity

### Setup

In [ ]:
# Grid size
m = 1024
u = np.empty((m))
v = np.empty((m))
snaps = 64
iters = 200
z = np.empty((m, snaps+1))

# Initial condition
def f(x):
    return exp(-2-2*cos(2*pi*x))


# PDE-related constants
A = 1.0
epsilon = 10**(-2.5)
dx = 1.0/m
dt = 0.06*dx

# Set up initial condition
for i in range(m):
    u[i] = f(dx*i)
z[:, 0] = u

# Define ENO2 operation
def eno2(a, b, c, d):
    if abs(a-2*b+c) < abs(b-2*c+d):
        return 3*c-4*b+a
    else:
        return d-b

### Integrate using ENO2

In [ ]:
# Integrate the equation
f = 0.5*A*dt/dx
nu = epsilon*dt/(dx*dx)
for i in range(1, snaps+1):
    for k in range(iters):
        ua = np.roll(u, 2)
        ub = np.roll(u, 1)
        ud = np.roll(u, -1)

        # Compute update from viscous term
        v[:] = (1-2*nu)*u+nu*(ub+ud)

        # Compute nonlinear advection term using ENO2 method
        for j in range(m):
            v[j] -= f*(1-2*u[j])*eno2(ua[j], ub[j], u[j], ud[j])
        u, v = v, u
    z[:, i] = u

# Output results
results = []
for j in range(m):
    e = [str(j*dx)]
    for i in range(snaps+1):
        e.append(str(z[j, i]))
    results.append(" ".join(e))
    # print(" ".join(e))

### Visualize

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=150)
x = np.arange(m) * dx
line, = ax.plot([], [], color='tab:blue')
timestamp = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=12, va='top')
ax.set_xlim(0, 1)
ax.set_ylim(-0.05, 1.05)
ax.set_xlabel('$x$')
ax.set_ylabel('$q(x, t)$')

def init():
    line.set_data([], [])
    timestamp.set_text('')
    return line, timestamp

def animate(i):
    line.set_data(x, z[:, i])
    timestamp.set_text(f't = {i*dt*iters:.3f}')
    return line, timestamp

ani = FuncAnimation(fig, animate, frames=snaps+1, init_func=init, blit=True, interval=80)
HTML(ani.to_jshtml())

## REA method for the traffic equation

### Setup

In [ ]:
# Grid size
m = 256
u = np.empty((m))
F = np.empty((m))
v = np.empty((m))
snaps = 64
iters = 20
z = np.empty((m, snaps+1))

# Initial condition
def f(x):
    if x > 0.25 and x < 0.75:
        return 0.8
    else:
        return 0

# PDE-related constants
dx = 1.0/m
dt = 0.2*dx

# Set up initial condition
for i in range(m):
    u[i] = f(dx*i)
z[:, 0] = u

# Define flux function
def flux(x):
    return x*(1-x)

### Integrate

In [ ]:
# Integrate the equation
f = dt/dx
for i in range(1, snaps+1):
    for k in range(iters):

        # Compute fluxes by solving the Riemann problem
        ul = np.roll(u, 1)
        for j in range(m):
            if ul[j] >= 0.5 and u[j] < 0.5:
                F[j] = 0.25
            else:
                Fu = flux(u[j])
                Ful = flux(ul[j])
                if (Fu-Ful < 0 and u[j]-ul[j] > 0) or \
                   (Fu-Ful > 0 and u[j]-ul[j] < 0):
                    F[j] = Fu
                else:
                    F[j] = Ful

        # Compute update using the fluxes
        v[:] = u-f*(np.roll(F, -1)-F)
        u, v = v, u
    z[:, i] = u

# Output results
results = []
for j in range(m):
    e = [str(j*dx)]
    for i in range(snaps+1):
        e.append(str(z[j, i]))
    results.append(" ".join(e))
    # print(" ".join(e))

### Visualize

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=150)
x = np.arange(m) * dx
line, = ax.plot([], [], color='tab:blue')
timestamp = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=12, va='top')
ax.set_xlim(0, 1)
ax.set_ylim(-0.1, 0.9)
ax.set_xlabel('$x$')
ax.set_ylabel('$q(x, t)$')

def init():
    line.set_data([], [])
    timestamp.set_text('')
    return line, timestamp

def animate(i):
    line.set_data(x, z[:, i])
    timestamp.set_text(f't = {i*dt*iters:.3f}')
    return line, timestamp

ani = FuncAnimation(fig, animate, frames=snaps+1, init_func=init, blit=True, interval=80)
HTML(ani.to_jshtml())